In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

Installed package System.Linq.Async version 4.0.0

In [1]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

In [1]:
public async IAsyncEnumerable<(DateTimeOffset x, double bestFps, double worstFps, double averagefps)> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceTestScene_CreepSpawning_BeforePooling.clef");
    using(var stream = compactFile.OpenText())
    {//
        while(stream.Peek() > 0){
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            yield return (
                logevent.Timestamp,
                1/double.Parse(logevent.Properties["bestDuration"].ToString()),
                1/double.Parse(logevent.Properties["worstDuration"].ToString()),
                double.Parse(logevent.Properties["framesInSample"].ToString()) / double.Parse(logevent.Properties["sampleDuration"].ToString())
            );
        }
    }
}

public struct PlotDatum{
    public double x {get; set;}
    public double y0 {get; set;}
    public double y1 {get; set;}
    public double y2 {get; set;}
}

var readData = await GetBestFrameCount().ToListAsync();

// convert our data to format plot format
var firstTime = readData.First().x;
var dataOp1 = readData.Select(i => new PlotDatum
    {
        x = (i.x - firstTime).TotalSeconds, 
        y0 = i.worstFps, 
        y1 = i.bestFps,
        y2 = i.averagefps
    })
    .ToList();


// data close load and unload time of the scene might be incorrect and unrepresentitive.  Trim some data from begining and end.
var data = dataOp1
    .SkipWhile(i => i.x <= 5)
    .TakeWhile( i => i.x <= dataOp1.Last().x - 5);
    data

index,x,y0,y1,y2
0,6.0055038,115.47210771238207,296.41925539483043,223.5
1,8.0094298,105.38629345867277,268.7160745955823,223.5
2,10.0146408,43.4769376584191,264.9497920144133,216
3,12.0151991,52.157222732203955,246.49970420035496,197
4,14.019564,92.98865538404316,244.0989088778773,197.5
5,16.0228165,91.05809506465124,238.16328474802324,178.5
6,18.0272589,40.28164929184861,223.01516503122212,185.5
7,20.0312095,56.68034552338631,219.59199806759042,190
8,22.035653,73.66591036332026,215.572993015435,171
9,24.0425793,36.183770131745106,200.81934291910997,167


In [1]:
#!javascript
notebookScope.plot = (sgvSelector, dataVariable) => {
    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });

    dtreeLoader(["d3"], function (d3) {
      var buildFrameRatePlot = (svg, width, height, data) => {
        var layoutMeasures = {
          width : width,
          height : height,
          plotPadding: [48,48,48,48], // top, right, bottom, left
          get plotWidth()
          {
            return layoutMeasures.width - layoutMeasures.plotPadding[1] - layoutMeasures.plotPadding[3]
          },
          get plotHeight()
          {
            return layoutMeasures.height - layoutMeasures.plotPadding[0] - layoutMeasures.plotPadding[2]
          }
        }
        
        // clean clear our svg
        svg.selectAll("g").remove();
      
        // plot container
        let container = svg
        .append("g")
        .attr("height", layoutMeasures.plotHeight)
        .attr("width", layoutMeasures.plotWidth)
        .attr("transform", `translate(${layoutMeasures.plotPadding[0]}, ${layoutMeasures.plotPadding[3]})`)
        .style("color", "black");
    
        container.append("rect")
          .attr("height", layoutMeasures.plotHeight)
          .attr("width", layoutMeasures.plotWidth)
          .style("fill", "#F7F7F7")
      
        var y = d3.scaleLinear()
          .domain([0, d3.max(data.flatMap(i => [i.y0, i.y1]))])
          .nice()
          .range([layoutMeasures.plotHeight, 0]);
        var x = d3.scaleLinear()
          .domain([0, d3.max(data, i => i.x)])
          .range([0, layoutMeasures.plotWidth])
        var yAxis = g => g
            .attr("transform",`translate(0,0)`)
            .call(d3.axisLeft(y))
        var xAxis = g => g
            .attr("transform",`translate(0,${layoutMeasures.plotHeight})`)
            .call(d3.axisBottom(x))
        container.append("g")
            .call(yAxis);
        container.append("g")
            .call(xAxis);
      
        // Line - best framerate in sample
        var linePoints_BestFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y0))
            (data);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'red')
          .attr('stroke-width', 3)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_BestFramerate);
      
        // Line - worst framerate in sample
        var linePoints_WorstFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y1))
            (data);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'green')
          .attr('stroke-width', 3)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_WorstFramerate);
      
        // Area - difference between best and worst framerate
        var areaDifference = d3.area()
          .curve(d3.curveLinear)
          .defined(d => !isNaN(d.x))
          .defined(d => !isNaN(d.y0))
          .x(d => x(d.x))
          .y0(d => y(d.y0))
          .y1(d => y(d.y1));
        container.append("path")
          .datum(data)
          .attr("fill", "steelblue")
          .attr("d", areaDifference);

        // Line - average framerate in sample
        var linePoints_AverageFramerate = d3.line()
        .x(d => x(d.x))
        .y(d => y(d.y2))
        (data);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'yellow')
          .attr('stroke-width', 1)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_AverageFramerate);
          
      // Plot title
      svg.append('text')
        .text("Frame Rate Over Time")
        .attr('x', layoutMeasures.width / 2)
        .attr('y', 24)
        .attr('dominant-baseline', 'middle')
        .attr("text-anchor", "middle")
        .style('font-size', 24)
      
      // X-axis title
      container.append('text')
        .text('Time since scene start (seconds)')
        .attr('dominant-baseline', 'middle')
        .attr('text-anchor', 'middle')
        .attr('x', layoutMeasures.plotWidth / 2)
        .attr('y', layoutMeasures.plotHeight + 36)
      
      // Y-axis title
      container
        .append('g')
        .attr('transform', `translate(-36, ${layoutMeasures.plotHeight / 2})`)
        .append('text')
          .text('Frames per second')
          .attr('dominant-baseline', 'middle')
          .attr('text-anchor', 'middle')
          .attr('transform', 'rotate(-90)')
      
      // Legend
      var legendData = [
        {label: 'fps based on worst refresh', color: 'green'},
        [label: 'fps based on average refresh', color: 'yellow']
        {label: 'fps based on best refresh', color: 'red'}
      ]
      var legend =svg
        .append('g')
        .attr('transform',`translate(${layoutMeasures.width - layoutMeasures.plotPadding[1] - 240}, 0)`)
      var legendScale = d3
        .scaleOrdinal(legendData.map(i=>i.color))
        .domain(legendData.map(i=>i.label));
      legend.selectAll("boxes")
        .data(legendData)
        .enter()
        .append('rect')
          .attr('fill',i => i.color)
          .attr('y', (d,i) => i * 32)
          .attr('width', 50)
          .attr('height', 16);
      legend.selectAll("labels")
        .data(legendData)
        .enter()
        .append('text')
          .attr('y', (d,i) => i * 32 + 12)
          .attr('x', 120)
          .attr('text-anchor', 'middle')
          .style('font-size', 14)
          .text(d=>d.label);
    }

    var svg = d3.select(sgvSelector);
    console.log(svg);
    interactive.csharp.getVariable(dataVariable)
        .then( data => buildFrameRatePlot(svg, 1200, 400, data));
    });
}

In [1]:
#!html
<svg id="dataPlot1" width=1200 height=400  style="background-color:white"></svg>

#!js
notebookScope.plot("svg#dataPlot1", "data" );

In [1]:
#!html
